In [6]:
from dolfinx import mesh
from mpi4py import MPI

# Define mesh (replace with actual dimensions)
domain = mesh.create_rectangle(MPI.COMM_WORLD, [[0, 0], [100, 50]], [20, 10])


In [7]:
from dolfinx import fem
import numpy as np
from petsc4py import PETSc

V = fem.functionspace(domain, ("CG", 1))

# Time-dependent boundary condition: Sinusoidal loading
t = 0
omega = 1000  # Frequency in Hz
amp = 5.0     # Amplitude in mm

boundary_value = fem.Constant(domain, PETSc.ScalarType(0.0))  # Initial value

def update_boundary_value(t):
    boundary_value.value = amp * np.sin(omega * t)

# Apply BC to the lower edge
boundary_dofs = fem.locate_dofs_geometrical(V, lambda x: np.isclose(x[1], 0))
bc = fem.dirichletbc(boundary_value, boundary_dofs, V)


In [8]:
import torch

# Load trained NN model
nn_model = torch.load("trained_model.pth")

def neural_network_force(displacement, velocity):
    # Convert to PyTorch tensor
    input_data = torch.tensor([displacement, velocity], dtype=torch.float32)
    # Predict force
    return nn_model(input_data).detach().numpy()


ModuleNotFoundError: No module named 'torch'

In [9]:
dt = 1e-4  # Time step
u = fem.Function(V)  # Displacement field
v = fem.Function(V)  # Velocity field
a = fem.Function(V)  # Acceleration field

t_final = 0.003  # Total simulation time
while t < t_final:
    # Compute external force
    # f_ext = neural_network_force(u.vector[:], v.vector[:])

    update_boundary_value(t)
    
    # Update acceleration using explicit time integration
    a.vector[:] = (f_ext - K @ u.vector[:]) / M
    
    # Update velocity and displacement
    v.vector[:] += dt * a.vector[:]
    u.vector[:] += dt * v.vector[:]
    
    t += dt


NameError: name 'f_ext' is not defined

In [ ]:
from dolfinx.io import XDMFFile
xdmf = XDMFFile(MPI.COMM_WORLD, "solution.xdmf", "w")
xdmf.write_function(u)
xdmf.close()
